In [ ]:
import pandas as pd
import re
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
from matplotlib import colormaps
import numpy as np
import geopandas as gpd
import seaborn as sns

# Population Wages

### What is the poverty line in the US?
**Source:** [Assistant Secretary for Planning and Evaluation](https://aspe.hhs.gov/topics/poverty-economic-mobility/poverty-guidelines) **Year:** 2024

In [ ]:
df_pov_guid = pd.read_csv('Datasets/Poverty_Guidelines.csv')
df_pov_guid.head()

Data Cleaning:
* Shorten column names.
* Convert data entries to numeric values.
* Add 'National_PG' column for weighted average of poverty guidelines across US.
* Select only national guidelines.

In [ ]:
df_pov_guid.columns = ['Household Size', 'US_PG', 'Alaska_PG', 'Hawaii_PG']

In [ ]:
df_pov_guid['US_PG'] = df_pov_guid['US_PG'].str.extract(r'(\d+\,\d+)').replace(',', '', regex=True).astype(float)
df_pov_guid['Alaska_PG'] = df_pov_guid['Alaska_PG'].str.extract(r'(\d+\,\d+)').replace(',', '', regex=True).astype(float)
df_pov_guid['Hawaii_PG'] = df_pov_guid['Hawaii_PG'].str.extract(r'(\d+\,\d+)').replace(',', '', regex=True).astype(float)

In [ ]:
non_con = df_pov_guid[['Alaska_PG', 'Hawaii_PG']].sum(axis=1)
con = df_pov_guid['US_PG'] * 49
df_pov_guid['National_PG'] = (con + non_con) / 51

In [ ]:
df_pov_guid = df_pov_guid.iloc[:, 0:5:4]

In [ ]:
df_pov_guid.head()

In [ ]:
pov_value = df_pov_guid['National_PG'].min()

### What is the minimum wage across the US?
**Source:** [US Department of Labor](https://www.dol.gov/agencies/whd/state/minimum-wage/history) **Year:** 2023

In [ ]:
df_min_wage = pd.read_csv('Datasets/Minimum_Wage.csv')
df_min_wage.head()

Data Cleaning:
* Use 'State' as umbrella term to include US Territories.
* Change national minimum wage to 'NATIONAL.'
* Select only data from 2023.
* Remove codes from the 2023 columns (i.e. numberic values only).
* Create column of annual salary for a 40 hour work week.

In [ ]:
df_min_wage = df_min_wage.rename(columns={'State or other\njurisdiction': 'State'})

In [ ]:
df_min_wage = df_min_wage.replace('Federal (FLSA)', 'NATIONAL')

In [ ]:
df_min_wage = df_min_wage.iloc[:, 0:5:4]

In [ ]:
df_min_wage['2023'] = df_min_wage['2023'].str.extract(r'(\d+(?:\.\d+)?)').astype(float)

In [ ]:
df_min_wage['Annual Salary'] = df_min_wage['2023'] * 40 * 52

In [ ]:
df_min_wage.head()

In [ ]:
df_min_wage_ORD = df_min_wage.sort_values(by='Annual Salary').dropna()
plt.figure(figsize=(12, 4))
plt.bar(df_min_wage_ORD['State'], df_min_wage_ORD['Annual Salary'])
plt.axhline(pov_value, color='black',linestyle='--', label=f'Poverty Line')
plt.ylabel('Salary')
plt.title('Annual Salary Based on Minimum Wage')
plt.xticks(rotation=90)
plt.legend()
plt.show()

### What is the living wage across the US?
*Note: Median living wage is also avaiable.*

**Source:** [Economic Policy Institute](https://files.epi.org/uploads/fbc_livingwage_data_2024.xlsx) **Year:** 2024

In [ ]:
df_liv_wage = pd.read_csv('Datasets/Living_Wage.csv')
df_liv_wage.head()

Data Cleaning:
* Remove columm of state abbreviations.
* Reduce rows to include the 'standard' values.
* Reduce columns to include 'all income from wages' values.
* Combine information from first two rows.
* Use only data from one adult, no children.
* Convert data to numeric values.
* Create column with annual salary for a 40 hour work week.

In [ ]:
df_liv_wage = df_liv_wage.iloc[:, 1:]

In [ ]:
df_liv_wage = df_liv_wage[~df_liv_wage['Unnamed: 2'].str.contains('Median', na=False)]

In [ ]:
df_liv_wage = df_liv_wage.rename(columns={'Unnamed: 1': 'State'})
df_liv_wage.drop(df_liv_wage.columns[df_liv_wage.columns.str.contains('Unnamed|Summary')], axis=1, inplace=True)

In [ ]:
df_liv_wage = df_liv_wage[1:]

In [ ]:
df_liv_wage = df_liv_wage.iloc[:, :2]

In [ ]:
df_liv_wage['One adult, no children'] = df_liv_wage['One adult, no children'].str.extract(r'(\d+(?:\.\d+)?)').astype(float)

In [ ]:
df_liv_wage['Annual Salary'] = df_liv_wage['One adult, no children'] * 40 * 52

In [ ]:
df_liv_wage = df_liv_wage.reset_index(drop=True)
df_liv_wage.head()

In [ ]:
df_combined = pd.merge(df_min_wage, df_liv_wage, left_on='State', right_on='State')
df_combined.columns = ['State', 'Min Wage', 'MW Salary', 'Liv Wage', 'LW Salary']
df_combined.drop(df_combined.columns[~df_combined.columns.str.contains('Salary|State')], axis=1, inplace=True)
df_combined['LW - MW'] = df_combined['LW Salary'] - df_combined['MW Salary']
df_combined.head()

In [ ]:
df_combined_ORD = df_combined.sort_values(by='MW Salary').dropna()
plt.figure(figsize=(12, 4))
plt.bar(df_combined_ORD['State'], df_combined_ORD['MW Salary'], label='Min Wage Salary')
plt.bar(df_combined_ORD['State'], df_combined_ORD['LW - MW'], bottom=df_combined_ORD['MW Salary'], color='lightblue', label='Salary Gap')
plt.ylabel('Salary')
plt.title('Minimum Wage Vs. Living Wage')
plt.xticks(rotation=90)
plt.legend()
plt.show()

### What is the average (and median) income across the US?
**Source:** [US Census Bureau](https://data.census.gov/table/ACSST1Y2022.S1901?t=Income%20and%20Poverty&g=010XX00US,$0400000) **Year:** 2022

In [ ]:
df_salaries = pd.read_csv('Datasets/Average_Income.csv')
df_salaries.head()

Data Cleaning:
* Reduce columns to include only 'non-family' households, exlude margin of error columns.
* Reduce rows to include income brackets, median, and mean.
* Transpose data set.
* Name first column 'State,' median salary to 'Median Salary,' and mean salary to 'Average Salary.'
* Rename values in 'State' column to just the state.
* Rename 'United States' value to 'NATIONAL.'
* Convert data in 'Median Salary' and 'Average Salary' to numeric values.

In [ ]:
df_salaries.drop(df_salaries.columns[~df_salaries.columns.str.contains('Nonfamily households!!Estimate|Label')], axis=1, inplace=True)

In [ ]:
df_salaries = df_salaries[df_salaries['Label (Grouping)'].str.contains('Mean|Median', na=False)]

In [ ]:
df_salaries = df_salaries.set_index('Label (Grouping)').T.reset_index().rename_axis(None, axis=1)

In [ ]:
df_salaries = df_salaries.rename(columns={'index': 'State', 'Median income (dollars)': 'Median Salary', 'Mean income (dollars)': 'Average Salary'})

In [ ]:
df_salaries['State'] = df_salaries['State'].str.replace('!!Nonfamily households!!Estimate', '', regex=True)

In [ ]:
df_salaries = df_salaries.replace('United States', 'NATIONAL')

In [ ]:
df_salaries['Average Salary'] = df_salaries['Average Salary'].str.replace(',', '', regex=True).astype(float)
df_salaries['Median Salary'] = df_salaries['Median Salary'].str.replace(',', '', regex=True).astype(float)

In [ ]:
df_salaries.head()

In [ ]:
df_combined = pd.merge(df_combined, df_salaries, left_on='State', right_on='State')
df_combined.head()

In [ ]:
df_combined_ORD = df_combined.sort_values(by='MW Salary').dropna()
plt.figure(figsize=(12, 4))
plt.plot(df_combined_ORD['State'], df_combined_ORD['MW Salary'], color='black', label='Min Wage Salary')
plt.plot(df_combined_ORD['State'], df_combined_ORD['LW Salary'], color='black', linestyle='--', label='Liv Wage Salary')
plt.plot(df_combined_ORD['State'], df_combined_ORD['Median Salary'], label='Median Salary')
plt.plot(df_combined_ORD['State'], df_combined_ORD['Average Salary'], label='Average Salary')
plt.ylabel('Salary')
plt.title('Salary Information Across the US')
plt.xticks(rotation=90)
plt.legend()
plt.show()

# Teacher Wages

### What is the starting salary for teachers across the US?
**Source:** [National Education Association](https://www.nea.org/resource-library/educator-pay-and-student-spending-how-does-your-state-rank/starting-teacher) **Year:** FY 2022 - 2023

In [ ]:
df_star_sal = pd.read_csv('Datasets/Starting_Salary.csv')
df_star_sal.head()

Data Cleaning:
* Select data from starting salary and top salary columns.
* Convert data to numeric values.
* Rename columns in title case.
* Rename 'Dist. of Columbia' to 'District of Columbia.'
* Rename 'United States' to 'NATIONAL.'
* Remove '*' code from Texas.

In [ ]:
df_star_sal.drop(df_star_sal.columns[df_star_sal.columns.str.contains("'S")], axis=1, inplace=True)

In [ ]:
df_star_sal['STARTING SALARY'] = df_star_sal['STARTING SALARY'].str.extract(r'(\d+\,\d+)').replace(',', '', regex=True).astype(float)
df_star_sal['TOP SALARY'] = df_star_sal['TOP SALARY'].str.extract(r'(\d+\,\d+)').replace(',', '', regex=True).astype(float)

In [ ]:
df_star_sal.columns = ['State', 'Starting Salary', 'Top Salary']

In [ ]:
df_star_sal = df_star_sal.replace('United States', 'NATIONAL')
df_star_sal = df_star_sal.replace('Dist. of Columbia', 'District of Columbia')
df_star_sal = df_star_sal.replace('Texas*', 'Texas')

In [ ]:
df_star_sal.head()

In [ ]:
states = gpd.read_file('tl_2023_us_state/tl_2023_us_state.shp')
states = states.to_crs('EPSG:2267')
states = states.merge(df_star_sal, left_on='NAME', right_on='State')

alaska = states[states['NAME'] == 'Alaska']
hawaii = states[states['NAME'] == 'Hawaii']
mainland = states[(states['NAME'] != 'Alaska') & (states['NAME'] != 'Hawaii')]

vmin = states['Starting Salary'].min()
vmax = states['Starting Salary'].max()
norm = Normalize(vmin=vmin, vmax=vmax)
cmap = colormaps['Blues']

fig, ax_main = plt.subplots(1, 1, figsize=(18, 14))
ax_alaska = fig.add_axes([0.1, 0.15, 0.2, 0.2])  # [left, bottom, width, height]
ax_hawaii = fig.add_axes([0.25, 0.15, 0.15, 0.45])

mainland.plot(ax=ax_main, column='Starting Salary', cmap=cmap, norm=norm, legend=True, 
    legend_kwds={'shrink': 0.25, 'pad': 0, 'aspect': 10}, linewidth=0.8, edgecolor='0.8')
alaska.plot(ax=ax_alaska, column='Starting Salary', cmap=cmap, norm=norm, legend=False)
hawaii.plot(ax=ax_hawaii, column='Starting Salary', cmap=cmap, norm=norm, legend=False)

ax_main.set_axis_off()
ax_alaska.set_axis_off()
ax_hawaii.set_axis_off()

ax_main.set_title('Starting Salaries for Teachers \n 2022 - 2023', fontsize=16)
plt.show()

In [ ]:
df_combined = pd.merge(df_combined, df_star_sal, left_on='State', right_on='State')

### What is the average salary for teachers across the US?
**Source:** [National Education Association](https://www.nea.org/resource-library/educator-pay-and-student-spending-how-does-your-state-rank/teacher) **Year:** 2022 - 2023

In [ ]:
df_avg_sal = pd.read_csv('Datasets/Average_Salary.csv')
df_avg_sal.head()

Data Cleaning:
* Remove rank (and percent) columns.
* Rename columns to be title case and only years when applicable
* Convert data to numeric values.

In [ ]:
df_avg_sal.drop(df_avg_sal.columns[df_avg_sal.columns.str.contains('RANK|PERCENT')], axis=1, inplace=True)

In [ ]:
df_avg_sal.columns = ['State', '2021 - 2022', '2022 - 2023']

In [ ]:
df_avg_sal['2021 - 2022'] = df_avg_sal['2021 - 2022'].str.extract(r'(\d+\,\d+)').replace(',', '', regex=True).astype(float)
df_avg_sal['2022 - 2023'] = df_avg_sal['2022 - 2023'].str.extract(r'(\d+\,\d+)').replace(',', '', regex=True).astype(float)

In [ ]:
df_avg_sal.head()

In [ ]:
df_combined = pd.merge(df_combined, df_avg_sal, left_on='State', right_on='State')

In [ ]:
df_combined_ORD = df_combined.sort_values(by='LW Salary').dropna()
plt.figure(figsize=(12, 4))
plt.plot(df_combined_ORD['State'], df_combined_ORD['LW Salary'], label='Liv Wage Salary')
plt.plot(df_combined_ORD['State'], df_combined_ORD['Starting Salary'], color='black', label='Starting Salary')
plt.plot(df_combined_ORD['State'], df_combined_ORD['Average Salary'], color='black', linestyle='dashed', label='Average Salary')
plt.plot(df_combined_ORD['State'], df_combined_ORD['Top Salary'], color='black', linestyle='dotted', label='Top Salary')
plt.ylabel('Salary')
plt.title('Teacher Salaries VS Living Wage')
plt.xticks(rotation=90)
plt.legend()
plt.show()

# School Enrollment

### What is the school enrollment rate across the US?
**Source:** [United States Census Bureau](https://data.census.gov/table/ACSST1Y2022.S1401?t=Education&g=010XX00US,$0400000) **Year:** 2022

In [ ]:
df_enroll = pd.read_csv('Datasets/School_Enrollment.csv')
df_enroll.head(5)

Data Cleaning:
* Select rows that are '... year olds enrolled in school'
* Keep columms '... Percent!!Estimate'
* Transpose columns and rows.
* Shorten all column names.
* Reduce state data to include only the state.
* Change 'United States' to 'NATIONAL.'
* Change data to numeric.

In [ ]:
df_enroll = df_enroll[df_enroll['Label (Grouping)'].str.contains('year olds enrolled in school', na=False)]

In [ ]:
df_enroll.drop(df_enroll.columns[~df_enroll.columns.str.contains('Percent!!Estimate|Label')], axis=1, inplace=True)

In [ ]:
df_enroll = df_enroll.set_index('Label (Grouping)').T.reset_index().rename_axis(None, axis=1)

In [ ]:
df_enroll.columns = ['State', 'Ages: 3-4', 'Ages: 5-9', 'Ages: 10-14', 'Ages: 15-17', 'Ages: 18-19', 'Ages: 20-24', 'Ages: 25-34']

In [ ]:
df_enroll['State'] = df_enroll['State'].str.replace('!!Percent!!Estimate', '', regex=True)

In [ ]:
df_enroll = df_enroll.replace('United States', 'NATIONAL')

In [ ]:
df_enroll['Ages: 3-4'] = df_enroll['Ages: 3-4'].str.replace('%', '', regex=True).astype(float)
df_enroll['Ages: 5-9'] = df_enroll['Ages: 5-9'].str.replace('%', '', regex=True).astype(float)
df_enroll['Ages: 10-14'] = df_enroll['Ages: 10-14'].str.replace('%', '', regex=True).astype(float)
df_enroll['Ages: 15-17'] = df_enroll['Ages: 15-17'].str.replace('%', '', regex=True).astype(float)
df_enroll['Ages: 18-19'] = df_enroll['Ages: 18-19'].str.replace('%', '', regex=True).astype(float)
df_enroll['Ages: 20-24'] = df_enroll['Ages: 20-24'].str.replace('%', '', regex=True).astype(float)
df_enroll['Ages: 25-34'] = df_enroll['Ages: 25-34'].str.replace('%', '', regex=True).astype(float)

In [ ]:
df_enroll.head()

In [ ]:
df_heatmap = df_enroll
df_heatmap.columns = ['State', '3-4', '5-9', '10-14', '15-17', '18-19', '20-24', '25-34']
df_heatmap.set_index('State', inplace=True)
df_heatmap = df_heatmap.T
plt.figure(figsize=(16, 8))
sns.heatmap(df_heatmap, annot=False, cbar_kws={'label': 'Percent of Population in Age Range'}, linewidths=0.5, linecolor='white')
plt.title('School Enrollment by Age Range')
plt.xlabel('States')
plt.ylabel('Age Ranges')
plt.xticks(rotation=90)
# plt.yticks(rotation=0)
plt.show()

In [ ]:
# top_5_enro = df_enroll.nlargest(5, 'Ages: 3-4')
# bottom_5_enro = df_enroll.nsmallest(5, 'Ages: 3-4').sort_values(by='Ages: 3-4', ascending=False)

# barWidth = 0.15

# r1 = np.arange(len(top_5_min['State']))
# r2 = [x + barWidth for x in r1]
# r3 = [x + barWidth for x in r2]
# r4 = [x + barWidth for x in r3]
# r5 = [x + barWidth for x in r4]
# r6 = [x + barWidth for x in r5] 
# r7 = [x + barWidth for x in r6]

# fig, axs = plt.subplots(1, 2, figsize=(15, 6), sharey=True) 

# axs[0].bar(top_5_enro['State'], top_5_enro['Ages: 3-4'], color='lightblue', width=barWidth, label='Ages: 3-4')
# axs[0].bar(r2, top_5_enro['Ages: 5-9'], color='blue', width=barWidth, label='Ages: 5-9')
# axs[0].bar(r3, top_5_enro['Ages: 10-14'], color='lightgreen', width=barWidth, label='Ages: 10-14')
# axs[0].bar(r4, top_5_enro['Ages: 15-17'], color='pink', width=barWidth, label='Ages: 15-17')
# axs[0].bar(r5, top_5_enro['Ages: 18-19'], color='coral', width=barWidth, label='Ages: 18-19')
# # axs[0].bar(r6, top_5_enro['Ages: 20-24'], color='orange', width=barWidth, label='Ages: 20-24')
# # axs[0].bar(r7, top_5_enro['Ages: 25-34'], color='yellow', width=barWidth, label='Ages: 25-34')
# axs[0].set_title('States with Highest Enrollment')
# axs[0].set_xlabel('State')
# axs[0].set_ylabel('Enrollment')
# axs[0].set_xticks([r + barWidth / 2 for r in range(len(top_5_enro['State']))], top_5_enro['State'])
# axs[0].legend()

# axs[1].bar(bottom_5_enro['State'], bottom_5_enro['Ages: 3-4'], color='lightblue', width=barWidth, label='Ages: 3-4')
# axs[1].bar(r2, bottom_5_enro['Ages: 5-9'], color='blue', width=barWidth, label='Ages: 5-9')
# axs[1].bar(r3, bottom_5_enro['Ages: 10-14'], color='lightgreen', width=barWidth, label='Ages: 10-14')
# axs[1].bar(r4, bottom_5_enro['Ages: 15-17'], color='pink', width=barWidth, label='Ages: 15-17')
# axs[1].bar(r5, bottom_5_enro['Ages: 18-19'], color='coral', width=barWidth, label='Ages: 18-19')
# # axs[1].bar(r6, bottom_5_enro['Ages: 20-24'], color='orange', width=barWidth, label='Ages: 20-24')
# # axs[1].bar(r7, bottom_5_enro['Ages: 25-34'], color='yellow', width=barWidth, label='Ages: 25-34')
# axs[1].set_title('States with Lowest Enrollment')
# axs[1].set_xlabel('State')
# axs[1].set_ylabel('Enrollment')
# axs[1].set_xticks([r + barWidth / 2 for r in range(len(bottom_5_enro['State']))], bottom_5_enro['State'])
# axs[1].legend()

# fig.suptitle('School Enrollment Across US (by Percent)', fontsize=16)

# plt.tight_layout()

# plt.show()

In [ ]:
df_combined = pd.merge(df_combined, df_enroll, left_on='State', right_on='State')

# Summary

In [ ]:
df_combined.head()

In [ ]:
# top_5_WAGE = df_ALL_WAGE.sort_values(by='Min_Wage Salary').nlargest(5, 'Min_Wage Salary')
# bottom_5_WAGE = df_ALL_WAGE.sort_values(by='Min_Wage Salary').nsmallest(5, 'Min_Wage Salary').sort_values(by='Min_Wage Salary', ascending=False)

# barWidth = 0.2

# r1 = np.arange(len(top_5_WAGE['State']))
# r2 = [x + barWidth for x in r1]
# r3 = [x + barWidth for x in r2]
# r4 = [x + barWidth for x in r3]

# fig, axs = plt.subplots(1, 2, figsize=(15, 6), sharey=True) 

# axs[0].bar(r1, top_5_WAGE['Min_Wage Salary'], color='lightblue', width=barWidth, label='Min_Wage Salary')
# axs[0].bar(r2, top_5_WAGE['Liv_Wage Salary'], color='blue', width=barWidth, label='Liv_Wage Salary')
# axs[0].bar(r3, top_5_WAGE['Median Salary'], color='green', width=barWidth, label='Median Salary')
# axs[0].bar(r4, top_5_WAGE['Average Salary'], color='lightgreen', width=barWidth, label='Average Salary')
# axs[0].axhline(pov_value, color='red', linewidth=2, linestyle='--', label=f'Poverty Line')
# axs[0].set_title('Highest 5 Salaries Based on Minimum Wage')
# axs[0].set_xlabel('State')
# axs[0].set_ylabel('Salary')
# axs[0].set_xticks([r + barWidth / 2 for r in range(len(top_5_sal['State']))], top_5_sal['State'])
# axs[0].legend()

# axs[1].bar(r1, bottom_5_WAGE['Min_Wage Salary'], color='orange', width=barWidth, label='Min_Wage Salary')
# axs[1].bar(r2, bottom_5_WAGE['Liv_Wage Salary'], color='coral', width=barWidth, label='Liv_Wage Salary')
# axs[1].bar(r3, bottom_5_WAGE['Median Salary'], color='pink', width=barWidth, label='Median Salary')
# axs[1].bar(r4, bottom_5_WAGE['Average Salary'], color='purple', width=barWidth, label='Average Salary')
# axs[1].axhline(pov_value, color='red', linewidth=2, linestyle='--', label=f'Poverty Line')
# axs[1].set_title('Lowest 5 Salaries Based on Minimum Wage')
# axs[1].set_xlabel('State')
# axs[1].set_ylabel('Salary')
# axs[1].set_xticks([r + barWidth / 2 for r in range(len(bottom_5_sal['State']))], bottom_5_sal['State'])
# axs[1].legend()

# fig.suptitle('Comparisons of Minimum Wage, Living Cost, Median, & Average Salaries', fontsize=16)

# plt.tight_layout()

# plt.show()